# HOMEWORK 7


GROUP NUMBER:

GROUP MEMBERS that participated in the production of this file: Yixiao Zhao; Junyima; Xinyi Liu; Xiao Li

Comments:

In this Homework we use the time-series approach to testing cactor models


### 1. 

In order to proceed you need Microsoft Excel  file "https://www.dropbox.com/s/hez02fvhxymvaqi/Homework6.xls?dl=1". 

and the factors in "https://www.dropbox.com/s/9346pp2iu5prv8s/MonthlyFactors.csv?dl=1"

In this problem set you will evaluate and test asset pricing models using the time-series framework. You will use the Gibbons, Ross, Shanken (GRS) test discussed in class. The small difference is that now you will apply the GRS test to a multifacto model.

Before we had

$$T(1+SR_{Market}^2)^{-1}\alpha'\Sigma_\alpha\alpha$$

Now instead of having the sharpe ratio squared of the market in the denominator, we have the sharpe ratio squared of the tangency portfolio among the set of factors, that is instead of $SR_Market^2$ we have:

$$E[F]\Sigma_F^{-1} E[F] $$

where $\sqrt{E[F]\Sigma_F^{-1} E[F] }$  is the maximum sharpe ratio one can obtain by investing in the set of factors.Not that E[F] is a vector of the average factor returns, so with 3 factors this has dimension 3 by 1. Note further how the multivariate formular becomes the univariate one when there is only one factor!

So the GRS is


$$T(1+E[F]\Sigma_F^{-1} E[F])^{-1}\alpha'\Sigma_\alpha\alpha$$

The above file contains four spreadsheets:
1)	30 industry value-weight portfolio returns
2)	10 portfolios formed on short-to-intermediate term past returns (from t 2 to t 12), *note that these portfolios start in 01/1927 rather than 07/1926
3)	25 portfolios formed on size and BE/ME,
4)	And, a proxy for the market portfolio (RM-RF).



In [133]:
url="https://www.dropbox.com/s/hez02fvhxymvaqi/Homework6.xls?dl=1"
import pandas as pd
parser = lambda date: pd.to_datetime(date,format='%Y%m')
IndPortfolio=pd.read_excel(url, sheetname='Industry portfolios (VW)', skiprows=2,  index_col=0, 
                           parse_dates=True, date_parser=parser)
IndPortfolio=IndPortfolio/100
PastReturnP=pd.read_excel(url, sheetname='Past return portfolios', skiprows=8,  index_col=0, 
                           parse_dates=True, date_parser=parser)
PastReturnP=PastReturnP/100
BEME=pd.read_excel(url, sheetname='25 size and BEME portfolios', skiprows=2,  index_col=0, 
                           parse_dates=True, date_parser=parser)
BEME=BEME/100
Mkt_Rf=pd.read_excel(url, sheetname='Market, Rf', skiprows=1,  index_col=0, 
                           parse_dates=True, date_parser=parser)
Mkt_Rf=Mkt_Rf/100

In [134]:
url2="https://www.dropbox.com/s/9346pp2iu5prv8s/MonthlyFactors.csv?dl=1"
#parser2 = lambda date: pd.to_datetime(date,format='%d/%m/%Y')
Factors=pd.read_csv(url2,index_col=0,header = 'infer',na_values=-99.99)
Factors.index=pd.to_datetime(Factors.index,dayfirst=True,errors='ignore')
Factors=Factors/100

In [135]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Part I:  30 Value-Weight Industry portfolios

a)	Using the 30 value-weighted industry portfolios, Estimate time series regressions, 

$$Rp_t - Rf_t = a + \beta_{i,M} (RM_t - Rf_t)+\beta_{i,SMB} (SMB_t)+\beta_{i,HML} HmL_t + eit$$,

for each of the 30 industry portfolios. Perform the multivariate GRS test; please report both the GRS F-statistic and its p-value. 

b)	Are the any value or growth industries? Does it line up what you would expect? Explain.

c)	Relative to the CAPM model are the alphas higher or lower with the three factor model?

d) Relative to the CAPM model are the appraisal ratios higher or lower with the the three factor model?

e) How much more of the overall the variation in returns the addition of the two factors help explain relative to the CAPM?






### Answer here:

In [136]:
#  a.  Perform the multivariate GRS test; please report both the GRS F-statistic and its p-value. #

In [137]:
import datetime as dt
IndPortfolio['DateE']=IndPortfolio.index
IndPortfolio['Date'] = IndPortfolio.DateE.dt.to_period('M')
Factors['DateE']=Factors.index
Factors['Date'] = Factors.DateE.dt.to_period('M')
IndMerge1=pd.merge(IndPortfolio,Factors,'left',on='Date',suffixes=['_IP','_F'])
IndMerge1=IndMerge1.set_index(IndPortfolio.Date)
IndMerge1= IndMerge1.drop('DateE_IP', axis=1)
IndMerge1= IndMerge1.drop('DateE_F', axis=1)
IndMerge1.head()

Food    Beer   Smoke   Games   Books   Hshld   Clths    Hlth  \
Date                                                                      
1926-07  0.0056 -0.0519  0.0129  0.0293  0.1097 -0.0048  0.0808  0.0177   
1926-08  0.0259  0.2703  0.0650  0.0055  0.1001 -0.0358 -0.0251  0.0425   
1926-09  0.0116  0.0402  0.0126  0.0658 -0.0099  0.0073 -0.0051  0.0069   
1926-10 -0.0306 -0.0331  0.0106 -0.0476  0.0947 -0.0468  0.0012 -0.0057   
1926-11  0.0635  0.0729  0.0455  0.0166 -0.0580 -0.0054  0.0187  0.0542   

          Chems   Txtls ...    Rtail   Meals     Fin   Other    Date      RF  \
Date                    ...                                                    
1926-07  0.0814  0.0039 ...   0.0007  0.0187  0.0037  0.0520 1926-07  0.0022   
1926-08  0.0550  0.0814 ...  -0.0075 -0.0013  0.0446  0.0676 1926-08  0.0025   
1926-09  0.0533  0.0231 ...   0.0025 -0.0056 -0.0123 -0.0386 1926-09  0.0023   
1926-10 -0.0476  0.0100 ...  -0.0220 -0.0411 -0.0516 -0.0849 1926-10  0.0032   
1926-11  0.0520  0.0311 ...   0.0652  0.0433  0.0224  0.0400 1926-11  0.0031   

            MKT     SMB     HML  Mom  
Date                                  
1926-07  0.0318 -0.0230 -0.0287  NaN  
1926-08  0.0289 -0.0140  0.0419  NaN  
1926-09  0.0059 -0.0132  0.0001  NaN  
1926-10 -0.0292  0.0004  0.0051  NaN  
1926-11  0.0284 -0.0020 -0.0035  NaN  

[5 rows x 36 columns]

In [138]:
ExcessReturn=IndMerge1
ExcessReturn.iloc[:,0:30]=IndMerge1.iloc[:,0:30].subtract(IndMerge1['RF'],axis='index')
ExcessReturn.iloc[:,32:33]=IndMerge1.iloc[:,32:33].subtract(IndMerge1['RF'],axis='index')
ExcessReturn.head()

Food    Beer   Smoke   Games   Books   Hshld   Clths    Hlth  \
Date                                                                      
1926-07  0.0034 -0.0541  0.0107  0.0271  0.1075 -0.0070  0.0786  0.0155   
1926-08  0.0234  0.2678  0.0625  0.0030  0.0976 -0.0383 -0.0276  0.0400   
1926-09  0.0093  0.0379  0.0103  0.0635 -0.0122  0.0050 -0.0074  0.0046   
1926-10 -0.0338 -0.0363  0.0074 -0.0508  0.0915 -0.0500 -0.0020 -0.0089   
1926-11  0.0604  0.0698  0.0424  0.0135 -0.0611 -0.0085  0.0156  0.0511   

          Chems   Txtls ...    Rtail   Meals     Fin   Other    Date      RF  \
Date                    ...                                                    
1926-07  0.0792  0.0017 ...  -0.0015  0.0165  0.0015  0.0498 1926-07  0.0022   
1926-08  0.0525  0.0789 ...  -0.0100 -0.0038  0.0421  0.0651 1926-08  0.0025   
1926-09  0.0510  0.0208 ...   0.0002 -0.0079 -0.0146 -0.0409 1926-09  0.0023   
1926-10 -0.0508  0.0068 ...  -0.0252 -0.0443 -0.0548 -0.0881 1926-10  0.0032   
1926-11  0.0489  0.0280 ...   0.0621  0.0402  0.0193  0.0369 1926-11  0.0031   

            MKT     SMB     HML  Mom  
Date                                  
1926-07  0.0296 -0.0230 -0.0287  NaN  
1926-08  0.0264 -0.0140  0.0419  NaN  
1926-09  0.0036 -0.0132  0.0001  NaN  
1926-10 -0.0324  0.0004  0.0051  NaN  
1926-11  0.0253 -0.0020 -0.0035  NaN  

[5 rows x 36 columns]

In [139]:
Estimates=pd.DataFrame(ExcessReturn.iloc[:,0:30].mean())
Estimates.columns=['AvgReturns']
Estimates['SampleStd']=ExcessReturn.iloc[:,0:30].std()
Estimates.head(3)

AvgReturns  SampleStd
Food     0.007060   0.047815
Beer     0.009352   0.072937
Smoke    0.008880   0.058095

In [140]:
w = np.ones(3)/3
R=Factors.iloc[:,1:4].mean()
R=R.values

Cov = Factors.iloc[:,1:4].cov()
Varp= w@Cov@w
wmve=w/w.sum()
Remve = wmve.T@R
Covinv = np.linalg.inv(Cov)
w = Covinv@R
Stdmve=(wmve.T@Cov@wmve)**0.5
SPmve = (R.T@Covinv@R)**0.5

In [141]:
#select start of sample
startsample='1926'
# Here we initiate a new dataset to record the regression residuals
Epsilons=ExcessReturn[startsample:].iloc[:,0:30]*0
Estimates['alpha_ts']=Estimates['AvgReturns']*0
x= sm.add_constant(ExcessReturn.loc[startsample:,'MKT':'HML'])
for portfolios in ExcessReturn.columns[0:30]:
    y= ExcessReturn.loc[startsample:,portfolios]
    results= sm.OLS(y,x).fit()
    # we already estimate this, so we don't need to record again
    Estimates.set_value(portfolios,'alpha_ts',results.params[0])
    Estimates.set_value(portfolios,'MKTbeta',results.params[1])
    Estimates.set_value(portfolios,'SMBbeta',results.params[2])
    Estimates.set_value(portfolios,'HMLbeta',results.params[3])
    Epsilons[portfolios]=results.resid
    
# estimate the varaince-covariance matrix between residuals    
Sigma=Epsilons.cov() 
# sample size
T=Epsilons.shape[0]
# number of test assets
N=Epsilons.shape[1]


In [172]:
Estimates.head(3)

AvgReturns  SampleStd  alpha_ts   MKTbeta   SMBbeta   HMLbeta
Food     0.007060   0.047815  0.002233  0.760203 -0.119918  0.029752
Beer     0.009352   0.072937  0.002563  0.879454  0.236220  0.138799
Smoke    0.008880   0.058095  0.004751  0.657701 -0.184397  0.057746

In [142]:
#F#
Chi2=T/(1+SPmve**2)*(Estimates.alpha_ts.values @ np.linalg.inv(Sigma.values) @ Estimates.alpha_ts.values.T)
Chi2

92.868314358632276

In [143]:
#p#
import scipy.stats as stats
p=1-stats.chi2.cdf(Chi2,N)
p

2.3974561891293433e-08

In [144]:
#  b.  Are the any value or growth industries? Does it line up what you would expect? Explain. #

In [145]:
# yes there are, and it line up what I expected. The three-factor model are based on the value and growth stocks.

In [146]:
#  c.  Relative to the CAPM model are the alphas higher or lower with the three factor model? #

In [147]:
# The CAPM model has alphas lower than the three-factor model.

In [148]:
#  d.  Relative to the CAPM model are the appraisal ratios higher or lower with the the three factor model? #

In [149]:
# it is lower than the one with three-factor model.

In [150]:
#  e.  How much more of the overall the variation in returns the addition of the two factors help explain relative to the CAPM? #

In [151]:
# it helped explain about 20%.

### Part II:  10 Past Return portfolios

f)	Repeat parts a), c), d) and e) for the 10 past return portfolios.



 ### Answer here:

In [152]:
#  a.  Perform the multivariate GRS test; please report both the GRS F-statistic and its p-value. #

In [153]:
PastReturnP['DateE']=PastReturnP.index
PastReturnP['Date'] = PastReturnP.DateE.dt.to_period('M')
IndMerge2=pd.merge(PastReturnP,Factors,'left',on='Date',suffixes=['_Past','_F'])
IndMerge2=IndMerge2.set_index(PastReturnP.Date)
IndMerge2= IndMerge2.drop('DateE_Past', axis=1)
IndMerge2= IndMerge2.drop('DateE_F', axis=1)
IndMerge2.head()
ExcessReturn2=IndMerge2
ExcessReturn2.iloc[:,0:10]=IndMerge2.iloc[:,0:10].subtract(IndMerge2['RF'],axis='index')
#ExcessReturn2=ExcessReturn2.iloc[:,0:10]
ExcessReturn2.head()

Loser       2       3       4       5       6       7       8  \
Date                                                                      
1927-01 -0.0357 -0.0487  0.0241 -0.0057 -0.0066  0.0071  0.0052  0.0011   
1927-02  0.0723  0.0575  0.0807  0.0725  0.0354  0.0390  0.0255  0.0302   
1927-03 -0.0350 -0.0329 -0.0428 -0.0506 -0.0078 -0.0270  0.0180  0.0031   
1927-04  0.0128 -0.0312 -0.0271 -0.0159  0.0234 -0.0038  0.0180 -0.0084   
1927-05  0.0368  0.0409  0.0560  0.0285  0.0595  0.0562  0.0436  0.0629   

              9  Winner    Date      RF     MKT     SMB     HML     Mom  
Date                                                                     
1927-01 -0.0068 -0.0049 1927-01  0.0025  0.0019 -0.0056  0.0483  0.0044  
1927-02  0.0395  0.0670 1927-02  0.0026  0.0444 -0.0010  0.0317 -0.0201  
1927-03  0.0059  0.0517 1927-03  0.0030  0.0043 -0.0160 -0.0267  0.0359  
1927-04  0.0139  0.0524 1927-04  0.0025  0.0071  0.0043  0.0060  0.0419  
1927-05  0.0786  0.0594 1927-05  0.0030  0.0574  0.0141  0.0493  0.0301

In [154]:
Estimates2=pd.DataFrame(ExcessReturn2.iloc[:,0:10].mean())
Estimates2.columns=['AvgReturns']
Estimates2['SampleStd']=ExcessReturn2.iloc[:,0:10].std()
Estimates2.head(3)

AvgReturns  SampleStd
Loser    0.000207   0.098155
2        0.004093   0.080899
3        0.004662   0.070169

In [155]:
#select start of sample
startsample='1926'
# Here we initiate a new dataset to record the regression residuals
Epsilons=ExcessReturn2[startsample:].iloc[:,0:10]*0
Estimates2['alpha_ts']=Estimates2['AvgReturns']*0
x= sm.add_constant(ExcessReturn2.loc[startsample:,'MKT':'HML'])
for portfolios in ExcessReturn2.columns[0:10]:
    y= ExcessReturn2.loc[startsample:,portfolios]
    results= sm.OLS(y,x).fit()
    # we already estimate this, so we don't need to record again
    Estimates2.set_value(portfolios,'alpha_ts',results.params[0])
    Estimates2.set_value(portfolios,'MKTbeta',results.params[1])
    Estimates2.set_value(portfolios,'SMBbeta',results.params[2])
    Estimates2.set_value(portfolios,'HMLbeta',results.params[3])
    Epsilons[portfolios]=results.resid
    
# estimate the varaince-covariance matrix between residuals    
Sigma=Epsilons.cov() 
# sample size
T=Epsilons.shape[0]
# number of test assets
N=Epsilons.shape[1]


In [171]:
Estimates2.head(3)

AvgReturns  SampleStd  alpha_ts   MKTbeta   SMBbeta   HMLbeta
Loser    0.000207   0.098155 -0.015534  1.394254  0.489460  0.430171
2        0.004093   0.080899 -0.009275  1.240915  1.528035  0.388408
3        0.004662   0.070169 -0.007088  1.127198  1.253430  0.329231

In [156]:
#F#
Chi2=T/(1+SPmve**2)*(Estimates2.alpha_ts.values @ np.linalg.inv(Sigma.values) @ Estimates2.alpha_ts.values.T)
Chi2

214.21267416737203

In [157]:
#p#
import scipy.stats as stats
p=1-stats.chi2.cdf(Chi2,N)
p

0.0

In [158]:
#  c.  Relative to the CAPM model are the alphas higher or lower with the three factor model? #
# The CAPM model has alphas higher than the three-factor model.

In [159]:
#  d.  Relative to the CAPM model are the appraisal ratios higher or lower with the the three factor model? #

In [160]:
# They are higher than the alphas with three-factor model.

In [161]:
#  e.  How much more of the overall the variation in returns the addition of the two factors help explain relative to the CAPM? #

In [162]:
 # it helped explain about 50%.

### Part III:  25 Size and BE/ME portfolios

g)Repeat parts a), c), d) and e) for the 25 portfolios

h)What pattern emerges? Do you find this surprsing? Explain what do we learn.





 ### Answer here:

In [163]:
#  a.  Perform the multivariate GRS test; please report both the GRS F-statistic and its p-value. #

In [164]:
BEME['DateE']=BEME.index
BEME['Date'] = BEME.DateE.dt.to_period('M')
IndMerge3=pd.merge(BEME,Factors,'left',on='Date',suffixes=['_BEME','_F'])
IndMerge3=IndMerge3.set_index(BEME.Date)
IndMerge3= IndMerge3.drop('DateE_BEME', axis=1)
IndMerge3= IndMerge3.drop('DateE_F', axis=1)
IndMerge3.head()
ExcessReturn3=IndMerge3
ExcessReturn3.iloc[:,0:25]=IndMerge3.iloc[:,0:25].subtract(IndMerge3['RF'],axis='index')
#ExcessReturn3=ExcessReturn3.iloc[:,0:25]
ExcessReturn3.head()

Low       2       3       4    High   Low.1     2.1     3.1  \
Date                                                                      
1926-07  0.0356 -0.0063 -0.0216  0.0013  0.0183  0.0197  0.0220  0.0027   
1926-08 -0.0246 -0.0898  0.0219  0.0036  0.0815  0.0192 -0.0143  0.0376   
1926-09 -0.0644 -0.0053 -0.0643 -0.0187  0.0063 -0.0208 -0.0149  0.0085   
1926-10 -0.0894 -0.0407 -0.0599  0.0540 -0.0287 -0.0212 -0.0359 -0.0539   
1926-11  0.0316  0.0634  0.0195 -0.0501  0.0023  0.0260 -0.0268  0.0270   

            4.1  High.1 ...      2.4     3.4     4.4  High.4    Date      RF  \
Date                    ...                                                    
1926-07 -0.0180 -0.0055 ...   0.0587  0.0181  0.0289  0.0034 1926-07  0.0022   
1926-08  0.0021  0.0592 ...   0.0394  0.0176  0.0523  0.0751 1926-08  0.0025   
1926-09 -0.0327 -0.0117 ...   0.0342 -0.0013 -0.0098 -0.0266 1926-09  0.0023   
1926-10 -0.0837 -0.0164 ...  -0.0333 -0.0256 -0.0499 -0.0613 1926-10  0.0032   
1926-11  0.0435  0.0242 ...   0.0222  0.0121  0.0335  0.0225 1926-11  0.0031   

            MKT     SMB     HML  Mom  
Date                                  
1926-07  0.0318 -0.0230 -0.0287  NaN  
1926-08  0.0289 -0.0140  0.0419  NaN  
1926-09  0.0059 -0.0132  0.0001  NaN  
1926-10 -0.0292  0.0004  0.0051  NaN  
1926-11  0.0284 -0.0020 -0.0035  NaN  

[5 rows x 31 columns]

In [165]:
Estimates3=pd.DataFrame(ExcessReturn3.iloc[:,0:25].mean())
Estimates3.columns=['AvgReturns']
Estimates3['SampleStd']=ExcessReturn3.iloc[:,0:25].std()
Estimates3.head(3)

AvgReturns  SampleStd
Low    0.005715   0.123114
2      0.006985   0.098850
3      0.010100   0.090502

In [166]:
#select start of sample
startsample='1926'
# Here we initiate a new dataset to record the regression residuals
Epsilons=ExcessReturn3[startsample:].iloc[:,0:25]*0
Estimates3['alpha_ts']=Estimates3['AvgReturns']*0
x= sm.add_constant(ExcessReturn3.loc[startsample:,'MKT':'HML'])
for portfolios in ExcessReturn3.columns[0:25]:
    y= ExcessReturn3.loc[startsample:,portfolios]
    results= sm.OLS(y,x).fit()
    # we already estimate this, so we don't need to record again
    Estimates3.set_value(portfolios,'alpha_ts',results.params[0])
    Estimates3.set_value(portfolios,'MKTbeta',results.params[1])
    Estimates2.set_value(portfolios,'SMBbeta',results.params[2])
    Estimates3.set_value(portfolios,'HMLbeta',results.params[3])
    Epsilons[portfolios]=results.resid
    
# estimate the varaince-covariance matrix between residuals    
Sigma=Epsilons.cov() 
# sample size
T=Epsilons.shape[0]
# number of test assets
N=Epsilons.shape[1]

In [173]:
Estimates3.head(3)

AvgReturns  SampleStd  alpha_ts   MKTbeta   HMLbeta
Low    0.005715   0.123114 -0.011034  1.287273  0.434568
2      0.006985   0.098850 -0.007250  1.082518  0.235669
3      0.010100   0.090502 -0.004368  1.050503  0.520633

In [167]:
#F#
Chi2=T/(1+SPmve**2)*(Estimates3.alpha_ts.values @ np.linalg.inv(Sigma.values) @ Estimates3.alpha_ts.values.T)
Chi2

347.28900378639815

In [168]:
#p#
import scipy.stats as stats
p=1-stats.chi2.cdf(Chi2,N)
p

0.0

In [169]:
#  c.  Relative to the CAPM model are the alphas higher or lower with the three factor model? #

In [170]:
#  Alphas in CAPM are higher than those in three factor model.

In [ ]:
#  d.  Relative to the CAPM model are the appraisal ratios higher or lower with the the three factor model? #

In [ ]:
#  Appraisal ratios are much higher with the three-factor model.

In [ ]:
#  e.  How much more of the overall the variation in returns the addition of the two factors help explain relative to the CAPM? #

In [ ]:
#  About 70%

In [ ]:
#  h.  What pattern emerges? Do you find this surprsing? Explain what do we learn. #

In [ ]:
# as the number of stock increases, the alphas in three factor models become lower than those from CAPM. 
# We think that this means three factor model does better in estimation.